In [1]:
import csv
import pymorphy2
import re

import psycopg2
import sklearn
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator
from sklearn.pipeline import make_pipeline

import time
from tqdm import tqdm

import json

morph = pymorphy2.MorphAnalyzer(lang='ru')

Данные: https://huggingface.co/datasets/its5Q/yandex-q

In [2]:
data = pd.read_json('answers.jsonl', lines=True)
data = data[['question', 'answer']]

In [3]:
data.head()

,question,answer
0,"Как войти в Роблокс, если не работает верифика...",Никак но можно попробовать найти сайты которые...
1,Под чьей властью находились восточные города? ...,Восточные города находятся под властью людей к...
2,Сколько стоят хорошие окна?,"На цену лучше не ориентироваться, это тот случ..."
3,Можно ли заниматься любовью на Пасху?,"Конечно можно, пост прошел, теперь можно есть ..."
4,Почему у ноутбука очень часто включается венти...,Если у вас открыто 20 вкладок это особо не наг...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836810 entries, 0 to 836809
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   question  836810 non-null  object
 1   answer    836810 non-null  object
dtypes: object(2)
memory usage: 12.8+ MB


In [5]:
questions_data = data['question'].to_list()
answers = data['answer'].to_list()
answers_id=[] 
questions=[] 

In [6]:
transform=0

for i, answer in enumerate(tqdm(questions_data)):
    words=answer.split(' ')
    phrase=""
    for word in words:
        # каждое слово из вопроса приводим в нормальную словоформу
        word = morph.parse(word)[0].normal_form  
        # составляем фразу из нормализованных слов
    phrase = phrase + word + " "
       
    # Если длинна полученной фразы больше 0 добавляем ей в массив вопросов и массив кодов ответов
    if (len(phrase)>0):
        questions.append(phrase.strip())
        answers_id.append(i)
        transform=transform+1

100%|█████████████████████████████████████████████████████████████████████████| 836810/836810 [26:54<00:00, 518.41it/s]


In [7]:
print(len(questions))
print(len(answers))
print(len(answers_id))

836810
836810
836810


In [8]:
# Сохраним подготовленные данные
with open('prepared/questions.txt', 'w') as filehandle:
    json.dump(questions, filehandle)
with open('prepared/answers.txt', 'w') as filehandle:
    json.dump(answers, filehandle)
with open('prepared/answers_id.txt', 'w') as filehandle:
    json.dump(answers_id, filehandle)

In [9]:
# Векторизируем вопросы в матрицу 
vectorizer_q = TfidfVectorizer()
vectorizer_q.fit(questions)
matrix_big_q = vectorizer_q.transform(questions)
print ("Размер матрицы: ")
print (matrix_big_q.shape)

Размер матрицы: 
(836810, 116000)


In [10]:
# Трансформируем матрицу вопросов в меньший размер для уменьшения объема данных

if transform > 1000:
    transform = 1000

svd_q = TruncatedSVD(n_components=transform)
svd_q.fit(matrix_big_q)

# получим трансформированную матрицу
matrix_small_q = svd_q.transform(matrix_big_q)

print ("Коэффициент уменьшения матрицы: ")
print ( svd_q.explained_variance_ratio_.sum())

Коэффициент уменьшения матрицы: 
0.3667208517273901


Функция поиска ответа

In [11]:
def softmax(x):
    proba = np.exp(-x)
    return proba / sum(proba)

class NeighborSampler(BaseEstimator):
    def __init__(self, k=5, temperature=2.0):
        self.k=k
        self.temperature = temperature
    def fit(self, X, y):
        self.tree_ = BallTree(X)
        self.y_ = np.array(y)
    def predict(self, X, random_state=None):
        distances, indices = self.tree_.query(X, return_distance=True, k=self.k)
        result = []
        for distance, index in zip(distances, indices):
            result.append(np.random.choice(index, p=softmax(distance * self.temperature)))
        return self.y_[result]

ns_q = NeighborSampler()

ns_q.fit(matrix_small_q, answers_id) 
pipe_q = make_pipeline(vectorizer_q, svd_q, ns_q)

In [15]:
print("Пишите ваш вопрос, слова exit или выход для выхода")

request = ""

while request not in ['exit', 'выход']:
    # получим текст от ввода
    print('\n', 10*'*')
    request=input()
    
    # разберем фразу на слова
    words= re.split('\W',request)
    phrase=""
    for word in words:
        word = morph.parse(word)[0].normal_form  # морфируем слово вопроса в нормальную словоформу
        # Нормализуем словоформу каждого слова и соберем обратно фразу
        phrase = phrase + word + " "
        
    # запустим функцию и получим код ответа
    reply_id = int(pipe_q.predict([phrase.strip()]))
    # выведем текст ответа
    if request not in ['exit', 'выход']:
        print('-', answers[reply_id])
    else:
        print('Пока, заходи еще!')

Пишите ваш вопрос, слова exit или выход для выхода

 **********
привет
- Привет.
Как праздники прошли? Как день? Что нового за эти дни? Погода как? Ты не устала? Какую читаешь книгу? Давно спортом занималась? Планируешь что-то на этот год?

 **********
куда сходить на выходных?
- Как ни прискорбно об этом было бы говорить, но в Комсомольске-на-Амуре на сегодняшний момент ничего кроме деградации не происходит. Безвольная, ни к чему не приспособленная власть, кругом разруха и хаос, город в буквальном смысле слова рассыпается на глазах. Городские парки в ужаснейшем состоянии, на Дворцы Культуры тоже без слёз невозможно взглянуть, в центре города под балконы жилых домов стропилы подкладывают, чтобы они не обвалились.... Город "медленно" изживает себя.... Нет хозяина в городе, одни предприниматели, которым только распродать его и хочется...

 **********
а если поехать на море?
- Потому что это успокаивает и расслабляет. Приятные монотонные звуки, которые производят горящие полешки в камине 